# Example
use diamonds dataset to predict **price** in terms of other variables. For this, 
a. estimate the parameters
b. select the imporant input features 
c. write down the predictive model 

since price is practically continous therefore we need to apply linear regression:
to do the following tasks:
1.   parameter learning
2.   feature selection
3.   predictive model (use steps 1 and 2 and link function)
4.   prediction 
5.   Evaluate the performance of prediction (split the dataset into trainset and testset)

We can do Linear regression using 2 different ways:
1.  using statistical learning
2.  using sklearn

data modeling:
1. import the function of the model
2. create the model
3.  fit the model
4. prediction using the model

# Statistical way to recall GLM

In [114]:
#Import neccessary library
from statistics import mean
from statsmodels.api import GLM, add_constant, families
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Data Preparation

In [115]:
data = read_csv('dataset/diamonds.csv')

X = data.drop(['price', 'Unnamed: 0'], axis=1)  # Input variables
y = data['price']  # Output variable

OE = OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good', 'Ideal', 'Premium']])

data_cut_OE = OE.fit_transform(X[['cut']])
data_cut_DF = DataFrame(data_cut_OE)
data_cut_DF.columns = ['encoded cut']

OHE = OneHotEncoder(handle_unknown='ignore')

data_color_OHE = OHE.fit_transform(X[['color']])
data_color_DF = DataFrame(data_color_OHE.toarray())
data_color_DF.columns = OHE.get_feature_names_out()

data_clarity_OHE = OHE.fit_transform(X[['clarity']])
data_clarity_DF = DataFrame(data_clarity_OHE.toarray())
data_clarity_DF.columns = OHE.get_feature_names_out()

# Merging multiple DataFrames

In [116]:
X_binary = concat([data_color_DF, data_clarity_DF], axis=1) # Excluding data_cut_DF since it is ordinal
X_scalable = X[['carat', 'depth', 'table', 'length_mm', 'width_mm', 'depth_mm']]  # Orginal numeric columns
X_scalable = concat([X_scalable, data_cut_DF], axis=1)  # Including encoded ordinal columns

# Applying StandardScaler

In [117]:
X_scaled = StandardScaler().fit_transform(X_scalable)
X_scaled_DF = DataFrame(X_scaled)
X_scaled_DF.columns = X_scalable.columns

In [118]:
X_PREP = concat([X_scaled_DF, X_binary], axis=1)  # Prepared Data

# Data Modelling for feature selection / reduced model

In [119]:
X = add_constant(X_PREP)  # Ading constant column to handle bias; 
model_full = GLM(y, X, family=families.Gaussian(families.links.log())).fit()

In [120]:
pvalues = dict(model_full.pvalues)  # Identifying the columns to drop
drop_columns = []
for x in pvalues:
    if pvalues[x] >= 0.05:
        drop_columns.append(x)
# print(drop_columns)
X_red = X.drop(drop_columns, axis=1)

Predictive Model:  yhat=5.455245+0.001909*ZN+0.022373*INDUS+0.028275*RAD-0.002238*MEDV    

In [121]:
rmse_full = []
rmse_red = []
for i in range(20):
    X = add_constant(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    #*************Full Data Modelling*************
    
    # Creating the Linear Regression Model
    model_full = GLM(y_train, X_train, family=families.Gaussian(families.links.log())).fit()

    # Predicting using Test-set
    pred_full = model_full.predict(X_test)
    rmse_full.append(sqrt(mean_squared_error(y_test, pred_full)))
    
    #*************Reduced Data Modelling*************
    
    # Creating the Linear Regression Model
    X = add_constant(X_red)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    model_reduced = GLM(y_train, X_train, family=families.Gaussian(families.links.log())).fit()
    
    # Predicting using Test-set
    pred_red = model_reduced.predict(X_test)
    rmse_red.append(sqrt(mean_squared_error(y_test, pred_red)))

[mean(rmse_full), mean(rmse_red)]

[225.3099879525569, 228.72941658063053]

# Deployment phase 

In [122]:
X = add_constant(X_red)
model_dep = GLM(y, X, family=families.Gaussian(families.links.log())).fit()  # create the linear reg model using

# Predicting using testset

In [123]:
new = X.tail(1)
pred_red = model_dep.predict(new)
print(pred_red)
y.tail(1)

858    2824.675874
dtype: float64


858    2871
Name: price, dtype: int64

# Statistical & sklearn way to recall GLM

In [124]:
# Import neccessary library
from statistics import mean
from statsmodels.api import GLM, add_constant, families
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Data Preparation

In [125]:
data = read_csv('dataset/diamonds.csv')

X = data.drop(['price', 'Unnamed: 0'], axis=1)  # Input variables
y = data['price']  # Output variable

OE = OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good', 'Ideal', 'Premium']])
data_cut_OE = OE.fit_transform(X[['cut']])
data_cut_DF = DataFrame(data_cut_OE)
data_cut_DF.columns = ['encoded cut']

OHE = OneHotEncoder(handle_unknown='ignore')

data_color_OHE = OHE.fit_transform(X[['color']])
data_color_DF = DataFrame(data_color_OHE.toarray())
data_color_DF.columns = OHE.get_feature_names_out()

data_clarity_OHE = OHE.fit_transform(X[['clarity']])
data_clarity_DF = DataFrame(data_clarity_OHE.toarray())
data_clarity_DF.columns = OHE.get_feature_names_out()

# Merging multiple DataFrames

In [126]:
X_binary = concat([data_color_DF, data_clarity_DF], axis=1) # Excluding data_cut_DF since it is ordinal
X_scalable = X[['carat', 'depth', 'table', 'length_mm', 'width_mm', 'depth_mm']]  # Orginal numeric columns
X_scalable = concat([X_scalable, data_cut_DF], axis=1) # Including encoded ordinal columns

# Applying standard scalar

In [127]:
X_scaled = StandardScaler().fit_transform(X_scalable)
X_scaled_DF = DataFrame(X_scaled)
X_scaled_DF.columns = X_scalable.columns

In [128]:
X_PREP = concat([X_scaled_DF, X_binary], axis=1)  # Prepared Data

# Data Modelling for feature selection / reduced model

In [129]:
X = add_constant(X_PREP)  # Ading constant column to handle bias
model_full = GLM(y, X, family=families.Gaussian(families.links.log())).fit()

In [130]:
pvalues = dict(model_full.pvalues)  # Identifying the columns to drop
drop_columns = []
for x in pvalues:
    if pvalues[x] >= 0.05:
        drop_columns.append(x)
# print(drop_columns)
X_red = X.drop(drop_columns, axis=1)

Predictive Model:  yhat=5.455245+0.001909*ZN+0.022373*INDUS+0.028275*RAD-0.002238*MEDV

In [131]:
rmse_st_full = []
rmse_st_red = []
rmse_skl_full = []
rmse_skl_red = []

model_skl = LinearRegression()
for i in range(20):
    #************************* Full Data Modelling  ***************************#
    X = add_constant(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # statsmodels
    model_st_full = GLM(y_train, X_train, family=families.Gaussian(families.links.log())).fit()

    # Predicting using Test-set
    pred_st_full = model_st_full.predict(X_test)
    rmse_st_full.append(sqrt(mean_squared_error(y_test, pred_st_full)))
    
    # sklearn
    model_skl.fit(X_train, y_train)
    
    # Predicting using Test-set
    pred_skl_full = model_skl.predict(X_test)
    rmse_skl_full.append(sqrt(mean_squared_error(y_test, pred_skl_full)))
    
    #************************* Reduced Data Modelling  ***************************#
    X = add_constant(X_red)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # statsmodels
    model_st_red = GLM(y_train, X_train, family=families.Gaussian(families.links.log())).fit()  # create the linear reg model using
    
    # Predicting using Test-set
    pred_red = model_reduced.predict(X_test)
    rmse_st_red.append(sqrt(mean_squared_error(y_test, pred_red)))
    
    # sklearn
    model_skl.fit(X_train, y_train)
    
    # Predicting using Test-set
    pred_skl_red = model_skl.predict(X_test)
    rmse_skl_red.append(sqrt(mean_squared_error(y_test, pred_skl_red)))
    

[mean(rmse_st_full), mean(rmse_st_red), mean(rmse_skl_full), mean(rmse_skl_red)]

[231.3826222352573, 221.00936319241126, 164.57963498013402, 158.71981793809638]

# Deployment Phase

In [132]:
X = add_constant(X_red)
model_skl_red_dep = model_skl.fit(X, y)

# Predicting using Test-set
# Predict the tax using the linear regression for the last row in the boston_house-prices dataset

In [133]:
prediction = model_skl_red_dep.predict(X.tail(1))
print(prediction)
y.tail(1)

[2809.03468253]


858    2871
Name: price, dtype: int64

# Logistic regression
Use boston_house price dataset, 
1.   Consider CHAS as the output variable 
2.   the remaining variables are inputs 
apply logistic regression to do the following tasks:
*  parameter estimation 
*   attribute selection 
*  predictive model
*   prediction

In [173]:
# Import neccessary library
from statistics import mean
from statsmodels.api import GLM, add_constant, families
from pandas import read_csv, DataFrame
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import accuracy_score, recall_score

# Data Preparation

In [174]:
data = read_csv('dataset/boston_house_prices.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [175]:
data['CHAS'].unique()

array([0, 1], dtype=int64)

In [176]:
X = data.drop('CHAS', axis=1)  # input
y = data['CHAS']   # output

# Data Modelling for feature selection / reduced model

# Logistic regression using statsmodels

In [177]:
X = add_constant(X)  # add intercept
model_st = GLM(y, X, family=families.Binomial()).fit()

In [178]:
pvalues = dict(model_st.pvalues)  # Identifying the columns to drop
drop_columns = []
for x in pvalues:
    if pvalues[x] >= 0.05:
        drop_columns.append(x)
# print("Columns to be romoved:", drop_columns)

In [179]:
X_red = X.drop(drop_columns, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=3)  # testset is 10%

Predictive model based on above results:

that_i = 0.0952 * INDUS_i + 0.2191 * RAD_i - 0.0087 * TAX_i + 0.0693 * MEDV_i

1.   phat_i = 1 / (1 + exp(-that_i))
2.   yhat_i = 1 if phat_i >= 0.5 else yhat_i = 0

In [180]:
model_st_red = GLM(y_train, X_train, family=families.Binomial()).fit() # logistic regression

In [181]:
prediction_st_raw = model_st_red.predict(X_test)

# Converting predictions to binary predicted values

In [182]:
prediction_st = [1 if prediction > 0.5 else 0 for prediction in prediction_st_raw]
df = DataFrame({"Prediction": prediction_st, "Actual": y_test})
df.head()


,Prediction,Actual
224,0,0
137,0,0
453,0,0
303,0,0
254,0,0


# Logistic regression using sklearn

In [183]:
model_skl = LogisticRegression(max_iter=10000).fit(X_train, y_train)
prediction_skl = model_skl.predict(X_test)

In [184]:
df = DataFrame({"Prediction_st": prediction_st, "Prediction_skl": prediction_skl, "Actual": y_test})
df.head()

,Prediction_st,Prediction_skl,Actual
224,0,0,0
137,0,0,0
453,0,0,0
303,0,0,0
254,0,0,0


# Compute recall and accuracy

In [185]:
# Prediction_st
acc_st = accuracy_score(y_test, prediction_st)
re_st = recall_score(y_test, prediction_st)
acc_skl = accuracy_score(y_test, prediction_skl)
re_skl = recall_score(y_test, prediction_skl)
print([acc_st, acc_skl])
[re_st, re_skl]


[0.8823529411764706, 0.8823529411764706]


[0.0, 0.0]

#Example

Use **winequality-red** dataset, apply NB classifier to predict **quality** variable. to do so, 

1. split the dataset into 70/30%

2. evaluate the model using **accurary** and **recall**.

3. validate the result using MC sampling with mc=100. 

4. predict the quality for a give set of input variables. 

5. redo the classification task using logistic regression and compare the result versus Naive Bayes classifier in 100 MC runs. Specify the best classifier

In [209]:
import pandas as pd
from pandas import read_csv
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import  GaussianNB
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score
from numpy import mean

In [210]:
dataset = pd.read_csv('dataset/winequality-red.csv')
X = dataset.drop('quality', axis=1)  # Input variables
y = dataset['quality']  # Output variable
# y.value_counts()

# Model evaluation

In [212]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)  # 70% training and 30% testgnb = GaussianNB()
gnb = GaussianNB().fit(X_train, y_train) # Train the model using the training sets

prediction = gnb.predict(X_test)  # Predict the response for test dataset
accuracy = accuracy_score(y_test, prediction)
recall = recall_score(y_test, prediction, average='micro')
[accuracy, recall]

[0.5416666666666666, 0.5416666666666666]

# Model validation using Monte Calro sampling

In [213]:
accuracy = []
for r in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    gnb = GaussianNB().fit(X_train, y_train)

    prediction = gnb.predict(X_test)  # Predict the response for test dataset
    accuracy.append(accuracy_score(y_test, prediction))
mean(accuracy)

0.5419291666666666

# Model validation based on Cross Validation score

In [214]:
gnb = GaussianNB()
scores = cross_val_score(gnb, X, y, cv=2)
mean(scores)

0.5278293178973718

# Prediction: for the last sample in the dataset

In [216]:
X_input = X.tail(1)
gnb.fit(X, y)
gnb.predict(X_input)

array([6], dtype=int64)

# Example 3: Use iris dataset
1.  Fit the Naive Bayes classifier  and evaluate the accuracy of the model in 100 mc runs. use 80% as the trainset and 20% testset.
2.  Recommend the type of flower for the following sample: X=[4,1,2,3].

In [2]:
from pandas import read_csv
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score
from numpy import mean

In [3]:
dataset = read_csv('dataset/Iris.csv')
X = dataset.drop('Species', axis=1)  # Input variables
y = dataset['Species']  # Output variable
# y.value_counts()

In [4]:
gnb = GaussianNB()
accuracy = []
for i in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
    gnb.fit(X_train, y_train)
    prediction = gnb.predict(X_test)
    accuracy.append(accuracy_score(prediction, y_test))
valid_acc = mean(accuracy)
valid_acc

0.9538666666666666

# 3-fold cross-validtion 
Validate the accuracy of GNB using 3 fold cross validation technique. 

In [5]:
cv = cross_val_score(gnb, X, y, cv=50)
mean(cv)

0.9533333333333335

# Model Deployment

In [6]:
model_gnb = gnb.fit(X.values, y.values) # Warning resoved
input = [[4, 1, 2, 3]]
model_gnb.predict(input)

array(['Iris-virginica'], dtype='<U15')

# Comparing with Logistic Regression

In [7]:
model_logistic = LogisticRegression(max_iter=10000) # Warning resolved
accuracy = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
    model_logistic.fit(X_train, y_train)
    prediction = model_logistic.predict(X_test)
    accuracy.append(accuracy_score(prediction, y_test))
valid_acc = mean(accuracy)
valid_acc

0.96

Example: predict **Species** in the **Iris** dataset using Support Vector Machine. 
*   train the model using 80% and evaluate it based on 20% testset
*   apply svm for different kernels
*   validate the result in 100 MC runs
*   Recommend the type of flower for the first sample in the dataset using the best classifier 

In [8]:
from pandas import read_csv
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from numpy import mean

In [9]:
dataset = read_csv('dataset/Iris.csv')
X = dataset.drop('Species', axis=1)  # Input variables
y = dataset['Species']  # Output variable

# svm for different kernels

In [10]:
accuracy = []
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
for i in kernel:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)  # testset is 30%
    model = SVC(kernel=i)
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    accuracy.append(accuracy_score(y_test, prediction))
accuracy

[0.9777777777777777,
 0.9555555555555556,
 0.9555555555555556,
 0.24444444444444444]

# Validate the result of performance for different kernels of svc in 100 MC runs

In [11]:
accuracy = []
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
for i in kernel:
    accuracy_mc = []
    for j in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=j)  # testset is 30%
        model = SVC(kernel=i, C=1.1)
        model.fit(X_train, y_train)
        prediction = model.predict(X_test)
        accuracy_mc.append(accuracy_score(y_test, prediction))  # append accuracy score in each MC run
    accuracy.append(mean(accuracy_mc)) # mean of accuracy and append it in accuracy in kernel array
accuracy

[0.9740000000000002, 0.964, 0.9593333333333333, 0.2515555555555555]

# 10-fold cross validation score 

In [12]:
accuracy = []
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
for i in kernel:
    model = SVC(kernel=i)
    cross_v_score = cross_val_score(model, X, y, scoring='accuracy', cv=10) # cv=10: 10-fold cross validation
    # mean of accuracy and append it in accuracy in kernel array
    accuracy.append(mean(cross_v_score))
accuracy

[0.9733333333333334,
 0.9666666666666668,
 0.9733333333333334,
 0.06666666666666668]

# Repeat the above expriment for NB classifier and logistic regression 

In [13]:
accuracy_svc = []
accuracy_gnb = []
accuracy_lr = []

model_lr = LogisticRegression(max_iter=10000)
model_gnb = GaussianNB()

cross_v_score_lr = cross_val_score(model_lr, X, y, scoring='accuracy', cv=6)
cross_v_score_gnb = cross_val_score(gnb, X, y, scoring='accuracy', cv=6)
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
for i in kernel:
    model_svc = SVC(kernel=i)
    cross_v_score_svc = cross_val_score(model_svc, X, y, scoring='accuracy', cv=6)
    accuracy_svc.append(mean(cross_v_score_svc)) # mean of accuracy and append it in accuracy in kernel array
accuracy = [accuracy_svc, mean(cross_v_score_lr), mean(cross_v_score_gnb)]
accuracy

[[0.98, 0.96, 0.98, 0.16666666666666666],
 0.9666666666666667,
 0.9533333333333333]

# Use Gini index to specify the root node of the decision tree for playtennis dataset. 

# Example 1: 
use decision tree algorithm with gini and entropy rules to predict the species of flowers in the Iris dataset, and compare the result versus SVC and Multinomial logistic regression in 1000 mc runs. 

In [20]:
from pandas import read_csv
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score
from numpy import mean
from sklearn.tree import DecisionTreeClassifier # decision tee algorithm for classification
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC

In [21]:
dataset = pd.read_csv('dataset/Iris.csv')
X = dataset.drop('Species', axis=1)  # matrix of input variables
y = dataset['Species']  # output variable

In [23]:
dt = DecisionTreeClassifier()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)  # split dataset
model_dt = dt.fit(X_train, y_train)  # fit the model
prediction = model_dt.predict(X_test)  # prediction
recall = recall_score(y_test, prediction, average='weighted')  # compute recall
recall
classification_report(y_test, prediction)


'                 precision    recall  f1-score   support\n\n    Iris-setosa       1.00      1.00      1.00        11\nIris-versicolor       0.89      1.00      0.94         8\n Iris-virginica       1.00      0.91      0.95        11\n\n       accuracy                           0.97        30\n      macro avg       0.96      0.97      0.96        30\n   weighted avg       0.97      0.97      0.97        30\n'

# Dictionary of Models

In [32]:
from pandas import read_csv
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score
from numpy import mean
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


def get_models():
    models = dict()
    models['dt_ent'] = DecisionTreeClassifier(criterion='entropy')
    models['dt_gini'] = DecisionTreeClassifier(criterion='gini')
    models['lr'] = LogisticRegression(max_iter=10000)
    models['svc_l'] = SVC(kernel='linear')
    models['svc_r'] = SVC()
    models['svc_s'] = SVC(kernel='sigmoid')
    models['svc_p'] = SVC(kernel='poly')
    return models

In [33]:
def evaluate_model(model, X, y, mc_run, split):
    accuracy = [] 
    for i in range(mc_run):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split)  # split dataset
        m = model.fit(X_train, y_train)  # fit the model
        prediction = m.predict(X_test)  # prediction
        accuracy.append(accuracy_score(y_test, prediction))  # compute & append accuracy
        return mean(accuracy)

In [40]:
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X, y, 100, 0.2)
    results.append(scores)
    names.append(name)
    print((name, mean(scores)))


('dt_ent', 0.9666666666666667)
('dt_gini', 0.9666666666666667)
('lr', 0.9666666666666667)
('svc_l', 1.0)
('svc_r', 0.9333333333333333)
('svc_s', 0.2)
('svc_p', 1.0)


# Evaluate the models in dictionary using k-fold cross validation

# Import Libraries

In [41]:
from pandas import read_csv
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score
from numpy import mean
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# get_models defination

In [42]:
def get_models():
    models = dict()
    models['dt_ent'] = DecisionTreeClassifier(criterion='entropy')
    models['dt_gini'] = DecisionTreeClassifier(criterion='gini')
    models['lr'] = LogisticRegression(max_iter=10000)
    models['svc_l'] = SVC(kernel='linear')
    models['svc_r'] = SVC()
    models['svc_s'] = SVC(kernel='sigmoid')
    models['svc_p'] = SVC(kernel='poly')
    return models

In [43]:
def cross_evaluator(model, X, y, k_fold):
    scores = cross_val_score(model, X, y, cv=k_fold)
    return mean(scores)

# Evulation

In [45]:
models = get_models() # Get the model list to be evaluated
results, names = list(), list()
print('The following result is the accuracy of different model in the dictionary of SML based on 6-fold cross validation')
for name, model in models.items():
    scores = cross_evaluator(model, X, y, 6)
    results.append(scores)
    names.append(name)
    print((name, mean(scores)))
# plot model performance for comparison


The following result is the accuracy of different model in the dictionary of SML based on 6-fold cross validation
('dt_ent', 0.9533333333333333)
('dt_gini', 0.9533333333333333)
('lr', 0.9666666666666667)
('svc_l', 0.98)
('svc_r', 0.98)
('svc_s', 0.16666666666666666)
('svc_p', 0.96)
